In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [153]:
import numpy as np

def unit_norm_layer_init(input_shape, output_shape):
    return np.random.normal(size=(input_shape, output_shape))


def ones_layer_init(input_shape, output_shape):
    return np.ones((input_shape, output_shape))


def zeros_init_layer(length):
    return np.zeros(length)


def softmax(x):
    # https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp, axis=1)[:, None]

def softmax_derivative(Z):
    N, K = Z.shape
    s = softmax(Z)[:, :, np.newaxis]
    a = np.tensordot(s, np.ones((1, K)), axes=([-1],[0]))
    I = np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0)
    b = I - np.tensordot(np.ones((K, 1)), s.T, axes=([-1],[0])).T
    return a * np.swapaxes(b, 1, 2)


def linear(x):
    return x


def linear_derivative(X):
    #np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0).shape
    return np.ones((X.shape[0]))

'''
def softmax_grad(x):
    s = sigmoid(x)
    _, K = s.shape
    return s @ (np.eye(K, K) - s).T
'''

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)


def ReLU(x):
    return np.maximum(x, 0, x)


def ReLU_derivative(x):
    return (x > 0).astype(int)


def cross_entropy(y_true, y_pred):
    N = len(y_true)
    return -np.sum(y_true * np.log(y_pred)) / N


def cross_entropy_derivative(y_true, y_pred):
    N = len(y_true)
    return -(y_true / y_pred) # / N


def one_hot_encode(y, n_classes):
    y_onehot = np.zeros((len(y), n_classes))
    for i, y_i in enumerate(y):
        y_onehot[i, y_i] = 1
    return y_onehot



def get_dE_dz(dE_da, da_dz):
    # array (N x K)
    # array (N x K x K)
    # output: array (N x K)
    N, K = dE_da.shape
    dE_dz = np.zeros((N, K))
    for n in range(N):
        dE_dz[n, :] = np.matmul(da_dz[n], dE_da[n, :, np.newaxis]).T
    return dE_dz


def normalize_trn_data(X):
    """
    normalize data to have zero mean and unit variance
    :param X: input data (array) - X.shape = (n_samples, m_features)
    :return:
    """
    mean, std = X.mean(axis=0), X.std(axis=0)
    return (X - mean) / std, (mean, std)


def shuffle_data(X, y):
    idx = np.arange(X.shape[0])
    np.random.shuffle(idx)
    return X[idx, :], y[idx]

def get_batch(X, y, batch_size):
    N, _ = X.shape
    batch_idxs = np.arange(0, N, batch_size)

    for start in batch_idxs:
        stop = start + batch_size
        X_batch, y_batch = X[start:stop], y[start:stop]
        yield X_batch, y_batch


class NeuralNetwork():
    
    def __init__(self, 
                 hidden=(8, 6),
                 init_weights='unit_norm',
                 init_bias='zeros',
                 activation='ReLU',
                 loss='cross_entropy',
                 mode='classification',
                 shuffle=True,
                 verbose=False,
                 batch_size=10,
                 random_state=1):
        self.hidden = hidden
        self.init_weights = init_weights
        self.init_bias = init_bias
        self.activation = activation
        self.loss = loss
        self.random_state = random_state
        self.mode = mode
        self.verbose = verbose
        self.shuffle = shuffle
        self.batch_size = batch_size
        np.random.seed(self.random_state)
        self._set_act_func()
        self._set_loss()
        
        
    def _init_neural_network(self):
        implemented_weight_inits = {'unit_norm': unit_norm_layer_init,
                                    'ones': ones_layer_init
                                   }
        implemented_bias_inits = {'zeros': zeros_init_layer,
                                   }
        try:
            init_layer_weight = implemented_weight_inits[self.init_weights]
            init_layer_bias = implemented_bias_inits[self.init_bias]
        except KeyError:
            raise Exception('{} or {} not accepted'.format(self.init_weights,
                                                           self.init_bias))

        self.weights = []
        self.biases = []
        for layer in range(len(self.hidden) + 1):
            if layer == 0:
                input_shape = self.n_features
                output_shape = self.hidden[layer]
            elif layer == len(self.hidden):
                input_shape = self.hidden[layer - 1]
                output_shape = self.n_classes
            else:
                input_shape = self.hidden[layer - 1]
                output_shape = self.hidden[layer]                
            w_l = init_layer_weight(input_shape, output_shape)
            b_l = init_layer_bias(output_shape)
            self.weights.append(w_l)
            self.biases.append(b_l)
        
            
    def _set_act_func(self):
        implemented_activations = {'sigmoid': sigmoid,
                                   'ReLU': ReLU,
                                   'linear': linear_derivative, 
                                    'softmax': softmax}
        # set activation function
        try:
            self.act = implemented_activations[self.activation]
        except KeyError:
            raise Exception('{} not accepted'.format(self.activation))
            
        implemented_derivatives = {'sigmoid': sigmoid_derivative,
                                   'ReLU': ReLU_derivative,
                                   'linear': linear_derivative,
                                    'softmax': softmax_derivative}
        
        # set activation derivative (da/dz)
        try:
            self.act_derivative = implemented_derivatives[self.activation]
        except KeyError:
            raise Exception('derivative not implemented for {}'.format(self.activation))

        # set activation for last layer (softmax for classification and linear for regression)
        if self.mode == 'classification':
            self.last_act = softmax
            self.last_act_grad = softmax_derivative
        elif self.mode == 'regression':
            self.last_act = linear
            self.last_act_grad = linear_derivative


    def _set_loss(self):
        implemented_losses = {'cross_entropy': cross_entropy,}
        loss_gradients = {'cross_entropy': cross_entropy_derivative,}
        try:
            self.loss_func = implemented_losses[self.loss]
            self.loss_grad_func = loss_gradients[self.loss]
        except KeyError:
            raise Exception('{} not accepted'.format(self.loss))
    
    def train(self, X, y, n_epochs=10, lr=0.001, n_classes=None):
        self.lr = lr
        self.n_samples, self.n_features = X.shape
        self.classes = n_classes
        if n_classes is None:
            self.classes = set(y)
            self.n_classes = len(self.classes)
        
        y_one_hot = one_hot_encode(y, self.n_classes)
        self._init_neural_network()
        
        self.loss_e = 0
        for e in range(n_epochs):
            
            if self.shuffle:
                X, y_one_hot = shuffle_data(X, y_one_hot)
                
            for X_batch, y_batch in get_batch(X, y_one_hot, self.batch_size):
                self._feed_forward(X_batch)
                self._back_prop(X_batch, y_batch)
                self.loss_batch = self.loss_func(y_batch, self.activations[-1])
                self.loss_e += self.loss_batch
        if self.verbose:
            print(e, 'trn loss = {}'.format(self.loss_e))
        print('epoch {}: final trn loss = {}'.format(e, self.loss_e))
            
    
    def _feed_forward(self, X):
        self.activations = []
        self.Z_list = []
        act = self.act
        for layer, (w_l, b_l) in enumerate(zip(self.weights, self.biases)):
            if layer == 0:
                prev = X
            else:
                prev = self.activations[-1]

            if layer == len(self.hidden):
                act = self.last_act
            Z_l = np.dot(prev, w_l) + b_l
            act_l = act(Z_l)    
            self.activations.append(act_l)
            self.Z_list.append(Z_l)
            
    def predict(self, X):
        self._feed_forward(X)
        return self.activations[-1]
            
    def _dE_dZ(self, y, p):
        # dE/dz where E(y) - cross entropy and a(z) is the softmax activation function
        return p - y
    
    def _get_grad(dE_da, da_dz):
        return np.tensordot(dE_da, da_dz, axes=([-1],[0]))
            
    def _back_prop(self, X, y):
        y_pred = self.activations[-1]
        z_last = self.Z_list[-1]
        self.dE_da = self.loss_grad_func(y, y_pred)
        self.da_dz = self.last_act_grad(z_last) # gradient of last activation layer
        self.dE_dz = get_dE_dz(self.dE_da, self.da_dz) # gradient at the last layer
        self.dE_dz_1 = self.dE_dz[...]
        self.dE_dz_2 = self._dE_dZ(y, y_pred)[...]

        new_weights, new_biases = [], []
        L = len(self.activations)
        for layer in range(L-1, -1, -1):
            w_l, b_l = self.weights[layer], self.biases[layer]
            Z_l = self.Z_list[layer]
            
            if layer == 0:
                act_prev = X
            else:
                act_prev = self.activations[layer-1]
            
            if layer < L-1:
                #self.dE_dz = self._dE_dZ(y, y_pred)
                dE_da = self.dE_dz @ self.weights[layer+1].T # dE_da wrt activation of current layer
                da_dz = self.act_derivative(Z_l)
                self.dE_dz = np.multiply(da_dz, dE_da)
            
            dE_dW = act_prev.T @ self.dE_dz
            dE_db = np.sum(self.dE_dz, axis=0)
            #print(layer, act_prev.T.shape, self.dE_dz.shape, dE_dW.shape, w_l.shape)
            w_l -= self.lr * dE_dW
            b_l -= self.lr * dE_db
            
            new_weights.append(w_l)
            new_biases.append(b_l)
            
        self.weights = new_weights[::-1]
        self.biases = new_biases[::-1]

In [154]:
# Load Iris Dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
iris = datasets.load_iris()
X = iris.data  
y = iris.target


feature_idxs = [1, 3] # SET FEATURES BY INDEX <------------------

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
xlbl, ylbl = feature_names[feature_idxs[0]], feature_names[feature_idxs[1]] 
# We will also split the dataset into training and testing so we can evaluate the kNN classifier
X_trn_, X_test_, y_trn, y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.333, 
                                                 random_state=0,
                                                 stratify=y)
X_trn, X_test = X_trn_[:, feature_idxs], X_test_[:, feature_idxs]

print("X_trn.shape = {}, X_test.shape = {}".format(X_trn.shape, X_test.shape))
X_trn_norm, (trn_mean, trn_std) = normalize_trn_data(X_trn)
X_test_norm = (X_test - trn_mean) / trn_std


nn = NeuralNetwork(hidden=(6,), 
                   init_weights='unit_norm', 
                   activation='ReLU',
                   shuffle=True,
                   batch_size=50,
                   random_state=1
                   )
nn.train(X_trn_norm, y_trn, n_epochs=100, lr=0.01)
y_pred_trn = nn.predict(X_trn_norm).argmax(axis=1)
y_pred_test = nn.predict(X_test_norm).argmax(axis=1)

print('trn acc', accuracy_score(y_pred_trn, y_trn))
print('test acc', accuracy_score(y_pred_test, y_test))

X_trn.shape = (100, 2), X_test.shape = (50, 2)
epoch 99: final trn loss = 36.444240410751206
trn acc 0.97
test acc 0.94


In [150]:
def softmax_gradient(z,sm=None):
    if sm is None:
        sm = softmax(z)
    res = np.einsum('ij,ik->ijk',sm,-sm)
    np.einsum('ijj->ij',res)[...] += sm
    return res

def dE_dz(y, z, sm=None):
    if sm is None:
        sm = softmax(z)
    dE_da = cross_entropy_derivative(y,sm)
    da_dz = da_dz_pp(z,sm)
    return np.einsum('ij,ijk->ik', dE_da, da_dz)

(array([1, 3, 2]), array([2, 4, 3]))

In [122]:
for e in range(10):
    XX, yy = shuffle_data(X_trn, y_trn)
    print(e)
    for xb, yb in get_batch(XX, yy, 30):
        print(xb[0], yb[0], xb.shape, yb.shape)
    print()

0
[2.3 1.3] 1 (30, 2) (30,)
[3.2 2.3] 2 (30, 2) (30,)
[2.7 1.9] 2 (30, 2) (30,)
[3.8 2.2] 2 (10, 2) (10,)

1
[2.2 1. ] 1 (30, 2) (30,)
[3.1 0.2] 0 (30, 2) (30,)
[2.8 2.4] 2 (30, 2) (30,)
[3.1 0.2] 0 (10, 2) (10,)

2
[2.6 1.2] 1 (30, 2) (30,)
[3.3 2.1] 2 (30, 2) (30,)
[3.2 1.8] 2 (30, 2) (30,)
[4.4 0.4] 0 (10, 2) (10,)

3
[3.  1.8] 2 (30, 2) (30,)
[2.9 1.3] 1 (30, 2) (30,)
[3.2 0.2] 0 (30, 2) (30,)
[2.9 1.3] 1 (10, 2) (10,)

4
[2.2 1.5] 2 (30, 2) (30,)
[3.  0.3] 0 (30, 2) (30,)
[3.  1.8] 2 (30, 2) (30,)
[3.2 0.2] 0 (10, 2) (10,)

5
[3.  1.8] 2 (30, 2) (30,)
[3.4 2.3] 2 (30, 2) (30,)
[2.9 1.4] 1 (30, 2) (30,)
[3.2 0.2] 0 (10, 2) (10,)

6
[3.4 0.4] 0 (30, 2) (30,)
[2.9 1.3] 1 (30, 2) (30,)
[3.  1.5] 1 (30, 2) (30,)
[2.7 1.6] 1 (10, 2) (10,)

7
[3.  1.8] 2 (30, 2) (30,)
[2.3 1.3] 1 (30, 2) (30,)
[2.9 1.3] 1 (30, 2) (30,)
[2.6 1.2] 1 (10, 2) (10,)

8
[2.9 1.3] 1 (30, 2) (30,)
[2.8 1.8] 2 (30, 2) (30,)
[2.7 1.9] 2 (30, 2) (30,)
[2.5 1.5] 1 (10, 2) (10,)

9
[3.  0.2] 0 (30, 2) (30,)
[3.3 2.1]

In [94]:
nn.dE_dz_1[0], nn.dE_dz_2[0]

(array([ 0.36015288, -0.68509994,  0.32494705]),
 array([ 0.36015288, -0.68509994,  0.32494705]))

In [44]:
def unit_norm_layer_init(input_shape, output_shape):
    return np.random.normal(size=(input_shape, output_shape))

def ones_layer_init(input_shape, output_shape):
    return np.ones((input_shape, output_shape))

def zeros_init_layer(length):
    return np.zeros(length)

def softmax(x):
    # https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp, axis=1)[:, None]

def linear(x):
    return x

def linear_derivative(X):
    return np.ones((X.shape[0]))

def softmax_grad(x):
    s = sigmoid(x)
    _, K = s.shape
    return s @ (np.eye(K, K) - s).T

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def ReLU(x):
    return np.maximum(x, 0, x)

def ReLU_derivative(x):
    return (x > 0).astype(int)

def cross_entropy(y_true, y_pred):
    N = len(y_true)
    return -np.sum(y_true*np.log(y_pred)) / N

def cross_entropy_derivative(y_true, y_pred):
    N = len(y_true)
    return -(y_true/y_pred) / N

def onehot_encode(y):
    y = np.array(y)
    y_onehot = np.zeros((len(y), max(y)+1))
    for i, y_i in enumerate(y):
        y_onehot[i, y_i] = 1
    return y_onehot

def one_hot_encode(y, n_classes):
    y_onehot = np.zeros((len(y), n_classes))
    for i, y_i in enumerate(y):
        y_onehot[i, y_i] = 1
    return y_onehot
'''
def softmax_grad(x):
    # da / dZ
    # Reshape the 1-d softmax to 2-d so that np.dot will do the matrix multiplication
    s = softmax(x).reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)
'''
class NeuralNetwork():
    
    def __init__(self, 
                 hidden=(8, 6),
                 init_weights='unit_norm',
                 init_bias='zeros',
                 activation='sigmoid',
                 loss='cross_entropy',
                 mode='classification',
                 random_state=1):
        self.hidden = hidden
        self.init_weights = init_weights
        self.init_bias = init_bias
        self.activation = activation
        self.loss = loss
        self.random_state = random_state
        self.mode = mode
        np.random.seed(self.random_state)
        self._set_act_func()
        self._set_loss()
        
        
    def _init_neural_network(self):
        implemented_weight_inits = {'unit_norm': unit_norm_layer_init,
                                    'ones': ones_layer_init
                                   }
        implemented_bias_inits = {'zeros': zeros_init_layer,
                                   }
        try:
            init_layer_weight = implemented_weight_inits[self.init_weights]
            init_layer_bias = implemented_bias_inits[self.init_bias]
        except KeyError:
            raise Exception('{} or {} not accepted'.format(self.init_weights,
                                                           self.init_bias))

        self.weights = []
        self.biases = []
        for layer in range(len(self.hidden) + 1):
            if layer == 0:
                input_shape = self.n_features
                output_shape = self.hidden[layer]
            elif layer == len(self.hidden):
                input_shape = self.hidden[layer - 1]
                output_shape = self.n_classes
            else:
                input_shape = self.hidden[layer - 1]
                output_shape = self.hidden[layer]                
            w_l = init_layer_weight(input_shape, output_shape)
            b_l = init_layer_bias(output_shape)
            self.weights.append(w_l)
            self.biases.append(b_l)
        
            
    def _set_act_func(self):
        implemented_activations = {'sigmoid': sigmoid,
                                   'ReLU': ReLU,
                                   'linear': linear_derivative}
        # set activation function
        try:
            self.act = implemented_activations[self.activation]
        except KeyError:
            raise Exception('{} not accepted'.format(self.activation))
            
        implemented_derivatives = {'sigmoid': sigmoid_derivative,
                                   'ReLU': ReLU_derivative,
                                   'linear': linear_derivative}
        
        # set activation derivative (da/dz)
        try:
            self.act_derivative = implemented_derivatives[self.activation]
        except KeyError:
            raise Exception('derivative not implemented for {}'.format(self.activation))

        # set activation for last layer (softmax for classification and linear for regression)
        if self.mode == 'classification':
            self.last_act = softmax
        elif self.mode == 'regression':
            self.last_act = linear

    def _set_loss(self):
        implemented_losses = {'cross_entropy': cross_entropy,}
        try:
            self.loss_func = implemented_losses[self.loss]
        except KeyError:
            raise Exception('{} not accepted'.format(self.loss))
    
    def train(self, X, y, n_epochs=10, lr=0.001, n_classes=None):
        self.lr = lr
        self.n_samples, self.n_features = X.shape
        self.classes = n_classes
        if n_classes is None:
            self.classes = set(y)
            self.n_classes = len(self.classes)
        
        y_one_hot = one_hot_encode(y, self.n_classes)
        self._init_neural_network()
        
        for e in range(n_epochs):
            # implement shuffle
            # implement batch
            self._feed_forward(X)
            self.loss_e = self.loss_func(y_one_hot, self.activations[-1])
            print('loss', e, self.loss_e)
            self._back_prop(X, y_one_hot)

    
    def _feed_forward(self, X):
        self.activations = []
        self.Z_list = []
        #self.Z_list.append(X)
        act = self.act
        for layer, (w_l, b_l) in enumerate(zip(self.weights, self.biases)):
            if layer == 0:
                prev = X
            else:
                prev = self.activations[-1]

            if layer == len(self.hidden):
                act = self.last_act
            Z_l = np.dot(prev, w_l) + b_l
            act_l = act(Z_l)    
            self.activations.append(act_l)
            self.Z_list.append(Z_l)
            
    def predict(self, X):
        self._feed_forward(X)
        return self.activations[-1]
            
    def _dE_dZ(self, y, p):
        # dE/dz where E(y) - cross entropy and a(z) is the softmax activation function
        return p - y
            
    def _back_prop(self, X, y):
        y_pred = self.activations[-1]
        z_last = self.Z_list[-1]
        
        da_dz = self.last_act_grad(z_last) # gradient of last activation layer
        # self.dE_dp = cross_entropy_derivative(y, y_pred)
        new_weights, new_biases = [], []
        L = len(self.activations)
        for layer in range(L-1, -1, -1):
            w_l, b_l = self.weights[layer], self.biases[layer]
            Z_l = self.Z_list[layer]
            
            if layer == 0:
                act_prev = X
            else:
                act_prev = self.activations[layer-1]
            
            if layer == L-1:
                self.dE_dz = self._dE_dZ(y, y_pred)
            else:
                dE_da = self.dE_dz @ self.weights[layer+1].T # dE_da wrt activation of current layer
                da_dz = self.act_derivative(Z_l)
                self.dE_dz = np.multiply(da_dz, dE_da)
            
            dE_dW = act_prev.T @ self.dE_dz
            dE_db = np.sum(self.dE_dz, axis=0)
            print(layer, act_prev.T.shape, self.dE_dz.shape, dE_dW.shape, w_l.shape)
            w_l -= self.lr * dE_dW
            b_l -= self.lr * dE_db
            
            new_weights.append(w_l)
            new_biases.append(b_l)
            
        self.weights = new_weights[::-1]
        self.biases = new_biases[::-1]
            #Z_l = self.Z_list[layer]
            #self.act_l = self.activations[layer]
            #prev_act = self.activations[layer-1]
            
            
            #self.da_dz = softmax_grad(Z_l)
            #if layer == 0: # last layer
            #    dE_dZ_l = dE_dyp # N vec X N vec

In [ ]:
# Load Iris Dataset
from sklearn import datasets
from sklearn.metrics import accuracy_score
iris = datasets.load_iris()
X = iris.data  
y = iris.target

# For illustration purposes we will only be using the two features in the dataset
feature_idxs = [1, 3] # SET FEATURES BY INDEX <------------------

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
xlbl, ylbl = feature_names[feature_idxs[0]], feature_names[feature_idxs[1]] 
# We will also split the dataset into training and testing so we can evaluate the kNN classifier
X_trn_, X_test_, y_trn, y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.333, 
                                                 random_state=0,
                                                 stratify=y)
X_trn, X_test = X_trn_[:, feature_idxs], X_test_[:, feature_idxs]

print("X_trn.shape = {}, X_test.shape = {}".format(X_trn.shape, X_test.shape))

In [ ]:

X = iris.data[:, :2]
y = iris.target
nn = NeuralNetwork(hidden=(8,6), 
                   init_weights='unit_norm', 
                   activation='ReLU',
                   )
nn.train(X_trn, y_trn, n_epochs=100, lr=0.0001)
y_pred_trn = nn.predict(X_trn).argmax(axis=1)
y_pred_test = nn.predict(X_test).argmax(axis=1)
print(accuracy_score(y_pred_trn, y_trn))
print(accuracy_score(y_pred_test, y_test))


In [ ]:
X = np.random.rand(100, 2)
y = np.random.randint(0, 2, size=100)

nn = NeuralNetwork(hidden=(8,6), 
                   init_weights='unit_norm', 
                   activation='ReLU',
                   )
nn.train(X, y, n_epochs=100, lr=0.001)



In [ ]:
X = np.random.rand(100, 2)
y = np.random.randint(0, 2, size=100)

nn = NeuralNetwork(hidden=(8,6), init_weights='unit_norm', activation='ReLU')
nn.train(X, y)

In [ ]:
X = [[1, 1], [0, 0], [0, 1]]
y = [1, 0, 1]

X, y = np.array(X), np.array(y)
nn = NeuralNetwork(hidden=(8,6), init_weights='unit_norm', activation='ReLU')
nn.train(X, y)
print(nn.loss_e)
for i in range(len(nn.activations)):
    
    print(i, nn.weights[i], '-', nn.biases[i],'-', nn.activations[i])
    
for i in range(len(nn.activations)):
    print(nn.weights[i].shape, '-', nn.biases[i].shape,'-', nn.activations[i].shape, '-', nn.Z_list[i].shape)

print(nn.dE_dz.shape)


In [ ]:
def cross_entropy_derivative(y_true, y_pred):
    # dE / da
    # input: N x K
    # output: N x K array
    N = len(y_true)
    return -(y_true / y_pred) / N

def softmax(x):
    # activation (a)
    # input: N x K array
    # output: N x K array
    # https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp, axis=1)[:, None]

def softmax_derivative(Z):
    # da/dz
    #input: N x K array
    #output: N x K x K array
    #http://saitcelebi.com/tut/output/part2.html
    s = softmax(Z)[:, :, np.newaxis]
    a = np.tensordot(s, np.ones((1, K)), axes=([-1],[0]))
    I = np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0)
    b = I - np.tensordot(np.ones((K, 1)), s.T, axes=([-1],[0])).T
    return a * np.swapaxes(b, 1, 2)

def softmax_derivative_test(Z):
    # da/dz
    # non-vectorized softmax gradient calculation
    #http://saitcelebi.com/tut/output/part2.html
    N, K = Z.shape
    da_dz = np.zeros((N, K, K))
    kron_delta = np.eye(K)
    s = softmax(Z)
    for n in range(N):
        for i in range(K):
            for j in range(K):
                da_dz[n, i, j] = s[n, i] * (kron_delta[i, j] - s[n, j])
    return da_dz


def dE_dz_test2(dE_da, da_dz):
    # array (N x K)
    # array (N x K x K)
    # output: array (N x K)
    N, K = dE_da.shape
    dE_dz = np.zeros((N, K))
    for n in range(N):
        dE_dz[n, :] = np.matmul(da_dz[n], dE_da[n, :, np.newaxis]).T
    return dE_dz
        
def some_type_of_matrix_multiplication_(dE_da, da_dz):
    # how do i get dE/dz from dE_da and da_dz
    pass

X = np.random.rand(100, 2)
W = np.random.rand(2, 4)
y = np.random.randint(0, 4, size=100)
y = one_hot_encode(y, 4)
Z = X @ W
S = softmax(Z)
N, K = Z.shape

# da / dz for softmax
da_dz = softmax_derivative(Z) # (100, 4, 4)
da_dz_test = softmax_derivative_test(Z) # (100, 4, 4) - non vectorized implementation
print(np.isclose(da_dz, da_dz_test).all()) # equivalence test

dE_da = cross_entropy_derivative(y, S) # (100, 4)
dE_dz = some_type_of_matrix_multiplication_(dE_da, da_dz) # what do I do here? *****
dE_dz_test  = (S - y) / N # (100, 4) If you combine dE/da and da/dz terms
dE_dz_test2 = dE_dz_test2(dE_da, da_dz)
print(np.isclose(dE_dz_test, dE_dz_test2).all()) # equivalence test


In [ ]:
da_dz.shape

In [ ]:
def dE_dz_test2(dE_da, da_dz):
    N, K = dE_da.shape
    dE_dz = np.zeros((N, K))
    for n in range(N):
        dE_dz[n, :] = np.matmul(da_dz[n], dE_da[n, :, np.newaxis]).T
    return dE_dz

In [ ]:
da_dz.shape, dE_da.shape

In [ ]:
dE_dz_test2_ = dE_dz_test2(dE_da, da_dz)
dE_dz_test[0], dE_dz_test2_[0]

In [ ]:
dE_dz_test_2 =  np.matmul(da_dz[0], dE_da[0, :, np.newaxis]).T
dE_dz_test[0], dE_dz_test_2

In [ ]:
np.tensordot(da_dz, dE_da[:, np.newaxis, :], axes=([1, 2],[1, 2]),  ).shape

#dz = np.matmul(matrix, da)

In [ ]:
da_dz.shape, dE_da[:, np.newaxis, :].shape

In [ ]:
dE_da[:, :, np.newaxis].shape

In [ ]:
dE_da.shape, da_dz.shape

In [ ]:
softmax_derivative_test(Z)[0], softmax_derivative(Z)[0]

In [ ]:
X = np.random.rand(100, 2)
W = np.random.rand(2, 4)
y = np.random.randint(0, 4, size=100)
y = one_hot_encode(y, 4)
Z = X @ W
N, K = Z.shape
s = softmax(Z)[:, :, np.newaxis]
dS = np.zeros((N, K, K))
#for n in range(N):
#    ds
    

In [ ]:

a = np.tensordot(s, np.ones((1, K)), axes=([-1],[0])) # np.matmul(s[:, :, np.newaxis], np.ones((1, K)))
I = np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0)
b = I - np.tensordot(np.ones((K, 1)), s.T, axes=([-1],[0])).T
c = a * np.swapaxes(b, 1, 2)
#np.matmul(np.ones((K, 1)), s[:, :, np.newaxis].T)
#b = np.identity(K) - a.T
#  matrix = np.matmul(a, np.ones((1, 3))) * (np.identity(3) - np.matmul(np.ones((3, 1)), a.T))
# dz = np.matmul(matrix, da)
c.shape

In [ ]:
# http://saitcelebi.com/tut/output/part2.html
a = np.tensordot(s[:, :, np.newaxis], np.ones((1, K)), axes=([-1],[0])) # np.matmul(s[:, :, np.newaxis], np.ones((1, K)))
I = np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0)
b = I - np.tensordot(np.ones((K, 1)), s[:, :, np.newaxis].T, axes=([-1],[0])).T
c = a * np.swapaxes(b, 1, 2)
#np.matmul(np.ones((K, 1)), s[:, :, np.newaxis].T)
#b = np.identity(K) - a.T
#  matrix = np.matmul(a, np.ones((1, 3))) * (np.identity(3) - np.matmul(np.ones((3, 1)), a.T))
c.shape

In [ ]:
c[0], tmp3

In [ ]:
b[0], tmp

In [ ]:
s.shape

In [ ]:
s2 = s[0]
I = np.eye(K)
tmp = I - np.matmul(np.ones((K, 1)), s2.T)
tmp2 = np.matmul(s2, np.ones((1, K)))
tmp3 = tmp2 * tmp
tmp3

In [ ]:
s2

In [ ]:
np.matmul(np.ones(K), s2.T)

In [ ]:
np.ones(K), s2.T

In [ ]:
s1 = np.sum(nn.dE_dz, axis=0)
np.ones(nn.dE_dz)

In [ ]:
?np.random.rand

In [ ]:
ds = 

In [ ]:
N, K = s.shape
I = np.repeat(np.eye(K, K)[np.newaxis, :, :], N, axis=0)
(I - s @ np.eye(K, K))

In [ ]:
tmp = np.multiply.outer(s, np.eye(1, K))

In [ ]:
s2 = s[0, : , np.newaxis]


In [ ]:
# http://saitcelebi.com/tut/output/part2.html

s0 = s[0, : , np.newaxis]
tmp = np.identity(K) - np.matmul(np.ones((K, 1)), s0.T)
tmp2 = np.matmul(s0, np.ones((1, K)))
tmp3 = tmp2 * tmp
tmp3

In [ ]:
np.matmul(np.ones((K, 1)), s0.T).shape

In [ ]:
np.ones((K, 1)).shape, s0.T.shape, np.matmul(np.ones((K, 1)), s0.T).shape

In [ ]:
I2 = np.ones((N, K))[:, : , np.newaxis]

print(I2.shape, s2.T.shape), 
print(np.tensordot(I2, s2.T, axes=([2],[0])).shape) #np.matmul(I2, s2.T).shape np.tensordot(I2, s2.T, axes=([2,1],[0,1])).shape

In [ ]:
np.matmul(I, s2.T).shape

In [ ]:
a = np.eye(2, 2)
print(a.shape)
# (2,  2)

# indexing with np.newaxis inserts a new 3rd dimension, which we then repeat the
# array along, (you can achieve the same effect by indexing with None, see below)


In [ ]:
y = [0, 1, 2, 1, 0]
y_ = np.zeros( (len(y), len(set(y))))
y_[:, y[:]] = 1
y_